In [ ]:
!pip install flask_cors
!pip install pydub
!pip install yt_dlp
!pip install pyngrok
!pip install googletrans==4.0.0-rc1
!pip install datasets
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok authtoken your-auth-token

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https

In [ ]:
from flask import Flask, request, jsonify
# from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from transformers import pipeline
from flask_cors import CORS
from pydub import AudioSegment
import yt_dlp
import torch
import os
import spacy
import re
import requests
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from IPython.display import display, Image
from datasets import Dataset

from googletrans import Translator

translator = Translator()

app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when the app runs

CORS(app)

# Initialize device and pipelines
device = "cuda:0" if torch.cuda.is_available() else "cpu"


asr_pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base",
    chunk_length_s=30,
    device=device,
    framework="pt"
)

summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device=device,
    framework="pt"
)

# Load NLP model for keyword extraction
nlp = spacy.load("en_core_web_sm")

def download_audio(video_url):
    ydl_opts = {
        'format': 'bestaudio',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav'}],
        'outtmpl': 'downloaded_audio'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])


def split_audio_into_chunks(audio_file_path, chunk_length_ms=60000, overlap_ms=5000):
    """
    Splits an audio file into overlapping chunks to avoid sentence cut-offs.

    Parameters:
    - audio_file_path (str): Path to the input audio file (WAV format).
    - chunk_length_ms (int): Length of each chunk in milliseconds (default: 60 seconds).
    - overlap_ms (int): Overlapping duration between chunks in milliseconds (default: 5 seconds).

    Returns:
    - List of chunk file paths.
    """
    audio = AudioSegment.from_wav(audio_file_path)
    total_length = len(audio)
    chunks = []

    start = 0
    chunk_id = 0

    while start < total_length:
        # Define the end position (ensure it doesn't exceed total length)
        end = min(start + chunk_length_ms, total_length)

        # Extract the chunk with overlap
        chunk = audio[start:end]

        # Export chunk to file
        chunk_path = f"chunk_{chunk_id}.wav"
        chunk.export(chunk_path, format="wav")
        chunks.append(chunk_path)

        # Move the start position, keeping overlap
        start += (chunk_length_ms - overlap_ms)
        chunk_id += 1

    return chunks


def dynamic_tokenizer_kwargs(input_length,length):
    max_length = min(1024, max(10, input_length // length))
    min_length = max(10, input_length // (2*length))
    return {'truncation': True, 'max_length': max_length, 'min_length': min_length}


def summarize_text(text, length):
    """Summarizes the given text after cleaning and chunking."""
    cleaned_sentences = remove_outliers(text)  # Remove unnecessary sentences
    full_transcription = " ".join(cleaned_sentences)

    summaries = []
    chunk_size = 300  # Summarization works best on ~300 words
    words = full_transcription.split()

    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        input_length = len(chunk.split())
        tokenizer_kwargs = dynamic_tokenizer_kwargs(input_length, length)
        summary = summarizer(chunk, do_sample=False, **tokenizer_kwargs)
        summaries.append(summary[0]['summary_text'])

    return " ".join(summaries), full_transcription


def generate_mind_map(summary_text):
    doc = nlp(summary_text)
    nodes = []
    links = []
    keyword_map = defaultdict(set)

    main_topics = set()

    # Extract key topics and relationships
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"]:  # Focus on nouns & proper nouns
            parent = token.head.text
            if parent != token.text:  # Avoid self-links
                keyword_map[parent].add(token.text)
                main_topics.add(parent)

    # Build nodes and links
    seen_nodes = set()
    for main_topic in main_topics:
        main_id = f"node-{main_topic}"
        if main_id not in seen_nodes:
            nodes.append({"id": main_id, "name": main_topic, "type": "heading"})
            seen_nodes.add(main_id)

        for sub in keyword_map[main_topic]:
            sub_id = f"node-{sub}"
            if sub_id not in seen_nodes:
                nodes.append({"id": sub_id, "name": sub, "type": "subtopic"})
                seen_nodes.add(sub_id)
            links.append({"source": main_id, "target": sub_id})

    return {"nodes": nodes, "links": links}

def extract_topic(text):
    doc = nlp(text)

    # Extract nouns and proper nouns
    words = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"] and not token.is_stop]

    # Count occurrences of each noun
    word_freq = Counter(words)

    # Prioritize words appearing in the first sentence(s)
    first_sentences = list(doc.sents)[:2]  # Look at the first two sentences
    first_mentions = [token.text for sent in first_sentences for token in sent if token.pos_ in ["NOUN", "PROPN"]]

    # Give extra weight to early mentions
    for word in first_mentions:
        word_freq[word] += 3  # Boost early mentions

    # Select the most frequently mentioned word
    main_topic = word_freq.most_common(1)

    return main_topic[0][0] if main_topic else "Unknown Topic"

def extract_sentences(summary):
    doc = nlp(summary)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

def extract_main_topics(text, top_n=10):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ in ['ORG', 'PERSON', 'GPE', 'LOC', 'PRODUCT']]
    entity_freq = Counter(entities)

    if entity_freq:
        main_topics = [topic for topic, _ in entity_freq.most_common(top_n)]
    else:
        words = [token.text for token in doc if token.is_alpha and not token.is_stop]
        word_freq = Counter(words)
        main_topics = [word for word, _ in word_freq.most_common(top_n)]

    return main_topics

def find_sentence_relations(keywords, sentences, max_sentences=1):
    keyword_sentences = defaultdict(list)
    assigned_sentences = set()  # To keep track of assigned sentences

    for keyword in keywords:
        for sentence in sentences:
            if len(sentence.split()) > 10 and sentence not in assigned_sentences:
                if keyword.lower() in sentence.lower():
                    if len(keyword_sentences[keyword]) < max_sentences:
                        keyword_sentences[keyword].append(sentence)
                        assigned_sentences.add(sentence)  # Mark sentence as used
                        break  # Stop after assigning the sentence to the keyword

    return keyword_sentences

def get_video_title(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.title.string.replace("- YouTube", "").strip()

def extract_most_frequent_title_word(title, text):
    doc = nlp(text.lower())  # Convert to lowercase for consistency
    words = [token.text for token in doc if token.is_alpha and not token.is_stop]  # Remove non-alphabetic words and stop words

    # Tokenize title words
    title_words = title.lower().split()

    # Count occurrences of title words in transcription
    word_freq = Counter(words)
    common_words = {word: word_freq[word] for word in title_words if word in word_freq}

    # Find the most frequent word
    return max(common_words, key=common_words.get, default="Unknown Title")

outro_patterns = [
    r"thanks for watching", r"see you in the next video", r"don't forget to subscribe",
    r"hit the (like|subscribe) button", r"leave a comment", r"hope you enjoyed",
    r"follow for more", r"stay tuned", r"this was all about", r"let me know in the comments",
    r"Welcome to this video", r"let's wrap up", r"in 100 seconds", r"this has been", r"for watching", r"video"
]

def remove_outliers(text):
    """ Removes unwanted sentences that match outro patterns. """
    sentences = list(nlp(text).sents)  # Segment into sentences
    filtered_sentences = []

    for sentence in sentences:
        sentence_text = sentence.text.strip()
        sentence_lower = sentence_text.lower()

        # Skip sentences matching engagement/outro phrases
        if any(re.search(pattern, sentence_lower) for pattern in outro_patterns):
            continue

        # Skip very short sentences (likely irrelevant)
        if len(sentence_text.split()) < 4:
            continue

        filtered_sentences.append(sentence_text)

    return filtered_sentences

def translate_text(text, target_language="es"):
    try:
        result = translator.translate(text, dest=target_language)
        return result.text
    except Exception as e:
        print("Translation Error:", str(e))
        return text  # Return the original text if translation fails




@app.route("/")
def home():
    return "Hello, Flask is running on Colab!"

@app.route("/summarize", methods=["POST"])
def summarize_video():
    try:
        data = request.get_json()
        video_url = data.get("video_url")
        if not video_url:
            return jsonify({"error": "Missing video_url"}), 400

        target_language = data.get("lang")
        if not target_language:
            return jsonify({"error": "Missing target language"}), 400

        # Determine summary length
        summary_length = data.get("summary_length", "Normal")
        length = {"Short": 3, "Normal": 2, "Long": 1}.get(summary_length, 2)

        # Download and split audio
        download_audio(video_url)
        audio_file_path = "downloaded_audio.wav"
        chunks = split_audio_into_chunks(audio_file_path, chunk_length_ms=60000, overlap_ms=5000)

        # Perform transcription
        dataset = Dataset.from_dict({"audio": chunks})

        # Process multiple audio chunks in a single batch
        def get_optimal_batch_size():
            total_vram = torch.cuda.get_device_properties(0).total_memory // (1024 ** 2)  # Get VRAM in MB
            print(total_vram)
            if total_vram >= 14000:  # T4 has 16GB
                return 48  # Safe batch size
            elif total_vram >= 12000:
                return 32
            else:
                return 24  # Safe fallback

        batch_size = get_optimal_batch_size()
        print(f"Using batch size: {batch_size}")

        transcriptions = asr_pipe(dataset["audio"], batch_size=batch_size)

        # Extract text and join into a single transcription
        transcription = " ".join([t["text"] for t in transcriptions])

        # Summarization
        full_summary, full_transcription = summarize_text(transcription, length)

        points = extract_sentences(full_transcription)
        text_data = "\n".join(points)
        # -------------------------------------------
        
        #---------------------------------------------------
        # Extract main topics (keywords)
        main_topics = extract_main_topics(text_data)

        # Determine central topic
        video_title = get_video_title(video_url)
        most_frequent_word = extract_most_frequent_title_word(video_title, full_transcription)

        # Find sentence relations for mind map
        keyword_sentences = find_sentence_relations(main_topics, points)

        # Translation (if needed)
        if target_language != "en":
            full_summary = translate_text(full_summary, target_language)
            full_transcription = translate_text(full_transcription, target_language)

        # Cleanup
        os.remove(audio_file_path)
        for chunk in chunks:
            os.remove(chunk)

        print(len(full_summary))


        return jsonify({
            "summary": full_summary,
            "transcription": full_transcription,
            "mind_map": keyword_sentences,
            "central": most_frequent_word
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    port = 5000
    public_url = ngrok.connect(port).public_url
    print("Public URL:", public_url)
    app.run(host="0.0.0.0",port=port)


Device set to use cuda:0
Device set to use cuda:0


Public URL: https://bfb8-35-185-199-89.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 19:25:26] "OPTIONS /summarize HTTP/1.1" 200 -


[youtube] Extracting URL: https://youtu.be/b9eMGE7QtTk?feature=shared
[youtube] b9eMGE7QtTk: Downloading webpage
[youtube] b9eMGE7QtTk: Downloading tv client config
[youtube] b9eMGE7QtTk: Downloading player 56511309
[youtube] b9eMGE7QtTk: Downloading tv player API JSON
[youtube] b9eMGE7QtTk: Downloading ios player API JSON
[youtube] b9eMGE7QtTk: Downloading m3u8 information
[info] b9eMGE7QtTk: Downloading 1 format(s): 251
[download] Destination: downloaded_audio
[download] 100% of   67.13MiB in 00:00:01 at 48.04MiB/s  
[ExtractAudio] Destination: downloaded_audio.wav
Deleting original file downloaded_audio (pass -k to keep)
15095
Using batch size: 48


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is

14670
